In [3]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

import os
import sys
from pprint import pprint as pp

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [54]:
"""Core geometry module for eigenbot, using signed distance function and probability graph."""

## EIGENGEOM-GTD 
# Obj: efficient shape PDFs from SDFs
#- SDF dataclass 
#- SDF - circle
#- SDF - box 
#- PDF dataclass
#- P(S) for circle, box
#- P(A) for circle, box

from dataclasses import dataclass
from typing import Union, Callable
import numpy.typing as npt
from numpy import typing as npt, linalg as LA   
import functools as ft 

dtype_vecnum = Union[npt.NDArray, float]
npa = lambda *args: np.array([*args])

@dataclass(init=True, repr=True, eq=True, order=False, frozen=True, 
           unsafe_hash=False)
class SDF:
    """Signed distance function (SDF) dataclass."""
    
    _sdf: Callable
    cx: dtype_vecnum
    cy: dtype_vecnum
    area: dtype_vecnum
    perim: dtype_vecnum
    
    def sdf(self, t, r):
        # If cross with unit-x is > 0 then convex, else concave. 
        _cpt_r = LA.norm(npa(self.cx, self.cy))  # length of cpt vector
        _cpt_cross2 = ((self.cx * 0.0) - (1.0 * self.cy)) / _cpt_r
        _is_convex = np.max(-_cpt_cross2, 0)
        # To handle convex angles, add rest of angle
        cpt_t = np.arccos(self.cx) 
        cpt_t = max(_is_convex * ((2 * np.pi) - cpt_t), cpt_t)
        return self._sdf(t, r) 
    
def sdf_circle(
    cx: dtype_vecnum, 
    cy: dtype_vecnum,
    rad: dtype_vecnum,
    ) -> Callable:
    """Returns SDF for circle given center (cx, cy), and rad radius.
    
    Derivation of SDF:
    .. math:: 
    
        \begin{align*}
            &r(x, y) \text{ is distance from circle center to (x, y). } \\
            &SDF(x, y)  &&= r(x, y) - rad \\
                        &&= ((x + cx)^2 + (y + cy)^2)^0.5 - rad \\
        \end{align*}
    
    """
    
    def _sdf_tr(t: dtype_vecnum, r:dtype_vecnum) -> npt.NDArray:
        # alt for params x, y: np.hypot(x + cx, y + cy) - rad
        # unit_xvec = npa(1.0, 0.0)  # theta=0
        # pvec = npa(x, y) / LA.norm(npa(x, y))
        # rcos = np.dot(pvec, unit_x)
        # rsin = np.cross(pvec, unit_x)
        # np.array([1+2j, 3+4j, 5+6j])
        # unit_xvec = npa(1+0j)
        # pvec = npa(np.exp(j * np.pi * t)  # exp(i pi t)
        return LA.norm(
            [(r * np.cos(t)) - cx, (r * np.sin(t)) - cy]
            ).astype(float) - rad

    return _sdf_tr

area_circle = lambda r: np.pi * r * r 
perim_circle = lambda r: 2 * np.pi * r
    
sdf = SDF(
    sdf_circle(0, 0, 1),
    cx=0, cy=0,
    area=np.pi, perim=2.0 * np.pi)
    
sdf.sdf(0, 2)

1.0

In [68]:
np.linalg.norm(npa(3+4j))




        

5.0


5.0

In [ ]:
# Test circle
assert abs( 0.0 - sdf_circle(0, 0, 1)(np.pi + np.pi/2.0, 1.0)) < 1e-6
assert abs( 1.0 - sdf_circle(0, 0, 2)(np.pi, 3.0)) < 1e-6
_1r_far = sdf_circle(100, 100, 1)
assert abs( 0.0 - _1r_far(np.pi/4, np.hypot(100, 100) + 1.0)) < 1e-6
# Test broadcasting
assert np.allclose(
    npa(-1, 0),
    (sdf_circle(npa(0, 1), npa(0, 1), npa(1, 2 ** 0.5))
     (np.pi / 4.0, 0)), 
    atol=1e-5)        
